# Getting started with ReColorAdv
This file contains instructions for experimenting with the ReColorAdv attack, by itself and combined with other attacks. This tutorial is based on the [first tutorial](https://github.com/revbucket/mister_ed/blob/master/notebooks/tutorial_1.ipynb) of `mister_ed`. See the README to make sure all dependencies are installed.

## Imports
First let's make sure that you can import everything you need:

In [ ]:
# EXTERNAL LIBRARIES
import numpy as np 
import scipy 

import torch
import torch.nn as nn 
import torch.optim as optim 

In [ ]:
# mister_ed
import recoloradv.mister_ed.loss_functions as lf 
import recoloradv.mister_ed.utils.pytorch_utils as utils
import recoloradv.mister_ed.utils.image_utils as img_utils
import recoloradv.mister_ed.cifar10.cifar_loader as cifar_loader
import recoloradv.mister_ed.cifar10.cifar_resnets as cifar_resnets
import recoloradv.mister_ed.adversarial_training as advtrain
import recoloradv.mister_ed.utils.checkpoints as checkpoints
import recoloradv.mister_ed.adversarial_perturbations as ap 
import recoloradv.mister_ed.adversarial_attacks as aa
import recoloradv.mister_ed.spatial_transformers as st

# ReColorAdv
import recoloradv.perturbations as pt
import recoloradv.color_transformers as ct
import recoloradv.color_spaces as cs
from recoloradv import norms

Now let's make sure that you have CIFAR-10 data loaded and a pretrained classifier running. If the following block fails, then make sure you've run the setup script:

```python -m recoloradv.mister_ed.scripts.setup_cifar```

In [ ]:
# Quick check to ensure cifar 10 data and pretrained classifiers are loaded 
cifar_valset = cifar_loader.load_cifar_data('val')
model, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, return_normalizer=True)

#  Generating adversarial examples

Here, we will demonstrate how to generate a single minibatch of adversarial examples using ReColorAdv on CIFAR-10.


To set up, let's start by collecting a minibatch worth of data and loading up our classifier to attack.

In [ ]:
cifar_valset = cifar_loader.load_cifar_data('val', batch_size=16)
examples, labels = next(iter(cifar_valset))

model, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, return_normalizer=True)

if utils.use_gpu():
    examples = examples.cuda()
    labels = labels.cuda() 
    model.cuda()

Let's take a look at what our original images look like:

In [ ]:
img_utils.show_images(examples)

## ReColorAdv
Now let's attack all of these examples with a ReColorAdv attack that changes every pixel using the same function.

In [ ]:
# This threat model defines the regularization parameters of the attack.
recoloradv_threat = ap.ThreatModel(pt.ReColorAdv, {
    'xform_class': ct.FullSpatial, 
    'cspace': cs.CIELUVColorSpace(), # controls the color space used
    'lp_style': 'inf',
    'lp_bound': [0.06, 0.06, 0.06],  # [epsilon_1, epsilon_2, epsilon_3]
    'xform_params': {
      'resolution_x': 16,            # R_1
      'resolution_y': 32,            # R_2
      'resolution_z': 32,            # R_3
    },
    'use_smooth_loss': True,
})


# Now, we define the main optimization term (the Carlini & Wagner f6 loss).
adv_loss = lf.CWLossF6(model, normalizer)

# We also need the smoothness loss.
smooth_loss = lf.PerturbationNormLoss(lp=2)

# We combine them with a RegularizedLoss object.
attack_loss = lf.RegularizedLoss({'adv': adv_loss, 'smooth': smooth_loss}, 
                                 {'adv': 1.0,      'smooth': 0.05},   # lambda = 0.05
                                 negate=True) # Need this true for PGD type attacks

# PGD is used to optimize the above loss.
pgd_attack_obj = aa.PGD(model, normalizer, recoloradv_threat, attack_loss)

# We run the attack for 10 iterations at learning rate 0.01.
perturbation = pgd_attack_obj.attack(examples, labels, num_iterations=10, signed=False, 
                                     optimizer=optim.Adam, optimizer_kwargs={'lr': 0.01},
                                     verbose=True)

# Now, we can collect the successful adversarial examples and display them.
successful_advs, successful_origs = perturbation.collect_successful(model, normalizer)
successful_diffs = ((successful_advs - successful_origs) * 3 + 0.5).clamp(0, 1)
img_utils.show_images([successful_origs, successful_advs, successful_diffs])

In the above image, the first row is the original images; the second row is the adversarial examples; and the third row is the magnified difference between them.

## Combined Attacks
Now that we've seen how to use the ReColorAdv attack, we can combine it with an additive delta attack.

In [ ]:
# First, we define the additive threat model.
additive_threat = ap.ThreatModel(ap.DeltaAddition, {
   'lp_style': 'inf', 
   'lp_bound': 0.03,
})

# Combine it with the ReColorAdv functional threat model.
combined_threat = ap.ThreatModel(
    ap.SequentialPerturbation, 
    [recoloradv_threat, additive_threat],
    ap.PerturbationParameters(norm_weights=[1.0, 0.0]),
)

# Again, define the optimization terms.
adv_loss = lf.CWLossF6(model, normalizer)
smooth_loss = lf.PerturbationNormLoss(lp=2)
attack_loss = lf.RegularizedLoss({'adv': adv_loss, 'smooth': smooth_loss}, 
                                 {'adv': 1.0,      'smooth': 0.05},
                                 negate=True) # Need this true for PGD type attacks

# Setup and run PGD over both perturbations at once.
pgd_attack_obj = aa.PGD(model, normalizer, combined_threat, attack_loss)
perturbation = pgd_attack_obj.attack(examples, labels, num_iterations=10, signed=False, 
                                     optimizer=optim.Adam, optimizer_kwargs={'lr': 0.01},
                                     verbose=True)

# Display the successful adversarial examples.
successful_advs, successful_origs = perturbation.collect_successful(model, normalizer)
successful_diffs = ((successful_advs - successful_origs) * 3 + 0.5).clamp(0, 1)
img_utils.show_images([successful_origs, successful_advs, successful_diffs])

Note that the resulting adversarial examples have been both recolored using ReColorAdv and had some additive adversarial noise applied from the delta attack.